<a href="https://colab.research.google.com/github/Lenzeg/Cogsci_blog/blob/master/thesis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thesis: Feature Engineering**
<small><i>Last update: 22 feb </i></small>


---


In this notebook I collect variables from both questionnaires and physiological data to use in a model that predicts depression progression.

The features put in the dataframe are for feature engineering purposes mainly and will be narrowed down to fewer features once links have been established.

The axis has been changed of the used data, meaning that instead of multiple rows for multiple timepoints, these values have been put as columns. This makes for easier statistical testing without having to do repeated measures analysis, which is also done because not all timepoints are relevant and most sleep features are only measured at T0.

\
**The aim to feature engineering for this project is:**


*   Establish the simplest form of predictability, and go complex from there. Logical interactions and ratios will be used based on literature or explained variance on the change of T0 and T1 depression scores or just T1 depression score. 
*   Have as few as possible features (degrees of freedom) that explain the most variance.



---


***TODO:***
Are we gonna include controls? Low risk? What is the exact question we're chasing?





**Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'


### **Import Variables from Questionnaires**

**ITQ** 
*   **ACS** - Action Control Scale
*   **BAS** - Behavioural System and Activation System Scale
*   **SHS**- Subjective Happiness Scale
*   **RPAS** - Response to Positive Affect
*   **RRS** - Ruminative Response Scale
*   **MINI-IPIP** - Mini International Personality Item Pool
*   **PANAS** - Positive and Negative Affect Scale
*   **TEPS** - Temporal Experience of Pleasure
*   **PI** - Perfectionism Inventory
*   **FIRST** - Ford Insomnia Response to Stress Test
*   **PSAS** - Pre-sleep Arousal Scale
*   **FSS** - Fatigue Severity Scale
*   **ISI** - Insomnia Severity Index
*   **CTQ** - Childhood Trauma Questionnaire

**Insomnia Subtypes**

0. Control
1. Highly Distressed
2. Moderately Distressed + Reward Sensitive
3. Moderately Distressed - Reward Sensitive
4. Low Distress + Highly Reactive
5. Low Distress + Low Reactive


**Actigraphy**

*   TIB - Time In Bed
*   TST - Total Sleep Time
*   SE - Sleep Efficiency 
*   SO - Sleep Onset
*   WASO - Wake After Sleep Onset
*   SOW - Sleep Opportunity Window
*   Mean_Temp - Mean night temperature   

In [3]:
# Import CSV's
ecg = pd.read_csv('ECG.csv', sep=";", usecols=['subject', 'HR_sleep','Drop.out'])
ids = pd.read_excel('IDS.xlsx', usecols=['subject', 'Male', 'Age', 'TimePoint', 'IDSSR_score','therapy_condition','risk','Drop.out'])
isi = pd.read_excel('ISI.xlsx', usecols=['subject', 'TimePoint', 'ISI_score','insomnia_subtype_true', 'Drop.out'])
screener = pd.read_excel('screener.xlsx', usecols=['subject', 'SCRN_ITQ_ACS_score','SCRN_ITQ_BAS_score','SCRN_ITQ_SHS_score', 'SCRN_ITQ_RPA_positive_rumination','SCRN_ITQ_RPA_dampening','SCRN_ITQ_RRS_score','SCRN_ITQ_MIPIP_extraversion','SCRN_ITQ_MIPIP_neuroticism','SCRN_ITQ_MIPIP_agreeableness','SCRN_ITQ_PANAS_negative_affect','SCRN_ITQ_PANAS_positive_affect','SCRN_ITQ_TEPS_score','SCRN_ITQ_PI_organization','SCRN_ITQ_PI_perceived_parental_pressure','SCRN_ITQ_PI_rumination','SCRN_ITQ_PI_score_R','SCRN_ITQ_FIRST_score','SCRN_ITQ_PSAS_score','SCRN_ITQ_FSS_score','SCRN_ITQ_CTQ_score','SCRN_ISI_score', 'Drop.out'])
actigraphy = pd.read_csv('actigraphy.csv', usecols=['subject', 'Drop.out', 'TimePoint','TST_TIB','TIB','SE_TIB','SOW','TST_SOW','Awakenings_TIB','Mean_Temp','Bedtime_Dec_Hr', 'EyesClosed_Dec_Hr','SleepOpEnd_Dec_Hr','Getuptime_Dec_Hr','Mid-Sleep_Time_Dec_Hr','Cr_of_Grav_Temp_Dec_Hr','Cr_of_Grav_to_Mid_Sleep_Delay_Dec_Hr'])
sleep = pd.read_csv('sleep.csv')
csd = pd.read_csv('220302_CSD_standard.csv')

# some processing before continuing
ids = ids.drop(ids[ids['Drop.out'] == 1].index)                       # dropping all dropouts
ids = ids.drop(ids[ids['therapy_condition'] == 5].index)              # 5 is not a treatment group
isi = isi.drop(isi[isi['Drop.out'] == 1].index)                       
ecg = ecg.drop(ecg[ecg['Drop.out'] == 1].index)
actigraphy = actigraphy.drop(actigraphy[actigraphy['Drop.out'] == 1].index)
screener = screener.drop(screener[screener['Drop.out'] == 1].index)             


# using mean of all recorded days for ecg features. HR_sleep serves as a temporary restless REM proxy
ecg_grouped = ecg[['subject','HR_sleep']].groupby(['subject']).mean().reset_index()   

# processing actigraphy features. std & mean for T0 and T1, group & rename.
act_sd_grouped = actigraphy[['subject','TimePoint','TST_TIB','TIB','SE_TIB','SOW','TST_SOW','Awakenings_TIB','Mean_Temp','Bedtime_Dec_Hr', 'EyesClosed_Dec_Hr','SleepOpEnd_Dec_Hr','Getuptime_Dec_Hr','Mid-Sleep_Time_Dec_Hr','Cr_of_Grav_Temp_Dec_Hr','Cr_of_Grav_to_Mid_Sleep_Delay_Dec_Hr']].groupby(['subject', 'TimePoint']).std().reset_index()  # using std of all recording days per timepoint
act_sd_T0 = act_sd_grouped.loc[(act_sd_grouped['TimePoint'] == 'T0')].add_prefix('T0_sd_').rename(columns={'T0_sd_subject':'subject','T0_sd_TimePoint':'TimePoint'})
act_sd_T1 = act_sd_grouped.loc[(act_sd_grouped['TimePoint'] == 'T1')].add_prefix('T1_sd_').rename(columns={'T1_sd_subject':'subject','T1_sd_TimePoint':'TimePoint'})

act_mean_grouped = actigraphy[['subject','TimePoint','TST_TIB','TIB','SE_TIB','SOW','TST_SOW','Awakenings_TIB','Mean_Temp','Bedtime_Dec_Hr', 'EyesClosed_Dec_Hr','SleepOpEnd_Dec_Hr','Getuptime_Dec_Hr','Mid-Sleep_Time_Dec_Hr','Cr_of_Grav_Temp_Dec_Hr','Cr_of_Grav_to_Mid_Sleep_Delay_Dec_Hr']].groupby(['subject', 'TimePoint']).mean().reset_index()   # using mean of all recorded days per timepoint
act_mean_T0 = act_mean_grouped.loc[(act_mean_grouped['TimePoint'] == 'T0')].add_prefix('T0_mean_').rename(columns={'T0_mean_subject':'subject','T0_mean_TimePoint':'TimePoint'})
act_mean_T1 = act_mean_grouped.loc[(act_mean_grouped['TimePoint'] == 'T1')].add_prefix('T1_mean_').rename(columns={'T1_mean_subject':'subject','T1_mean_TimePoint':'TimePoint'})

# processing csd features. std & mean for T0 and T1, group & rename.
csd_sd_grouped = csd[['subject','TimePoint','CSDM_TIB_Recalc','CSDM_SOW_Recalc','CSDM_TST_Recalc','CSDM_SEtib_Recalc','CSDM_SEsow_Recalc','CSD_Bedtime_Dec_Hr', 'CSD_EyesClosed_Dec_Hr','CSD_SleepOpEnd_Dec_Hr','CSD_SOL_min','CSD_LogSOL+1','CSD_WASO_num','CSD_logWASO_num+1','CSD_WASO_min','CSD_logWASO_min+1','CSD_EMA_min','logCSD_EMA_min+1','CSD_Getuptime_Dec_Hr','CSD_Mid-Sleep_Time_Dec_Hr','CSD_Qualit','CSD_WellRested','CSD_BedtimeSleepiness','CSD_OvernightSleepinessChange','CSD_Bedtime_Worry','CSD_Thoughts','CSD_Dreams','CSD_Thoughts-Dreams','CSD_Nap']].groupby(['subject', 'TimePoint']).std().reset_index()  # using std of all recording days per timepoint
csd_sd_T0 = csd_sd_grouped.loc[(csd_sd_grouped['TimePoint'] == 'T0')].add_prefix('T0_sd_').rename(columns={'T0_sd_subject':'subject','T0_sd_TimePoint':'TimePoint'})
csd_sd_T1 = csd_sd_grouped.loc[(csd_sd_grouped['TimePoint'] == 'T1')].add_prefix('T1_sd_').rename(columns={'T1_sd_subject':'subject','T1_sd_TimePoint':'TimePoint'})

csd_mean_grouped = csd[['subject','TimePoint','CSDM_TIB_Recalc','CSDM_SOW_Recalc','CSDM_TST_Recalc','CSDM_SEtib_Recalc','CSDM_SEsow_Recalc','CSD_Bedtime_Dec_Hr', 'CSD_EyesClosed_Dec_Hr','CSD_SleepOpEnd_Dec_Hr','CSD_SOL_min','CSD_LogSOL+1','CSD_WASO_num','CSD_logWASO_num+1','CSD_WASO_min','CSD_logWASO_min+1','CSD_EMA_min','logCSD_EMA_min+1','CSD_Getuptime_Dec_Hr','CSD_Mid-Sleep_Time_Dec_Hr','CSD_Qualit','CSD_WellRested','CSD_BedtimeSleepiness','CSD_OvernightSleepinessChange','CSD_Bedtime_Worry','CSD_Thoughts','CSD_Dreams','CSD_Thoughts-Dreams','CSD_Nap']].groupby(['subject', 'TimePoint']).mean().reset_index()  # using std of all recording days per timepoint
csd_mean_T0 = csd_mean_grouped.loc[(csd_mean_grouped['TimePoint'] == 'T0')].add_prefix('T0_mean_').rename(columns={'T0_mean_subject':'subject','T0_mean_TimePoint':'TimePoint'})
csd_mean_T1 = csd_mean_grouped.loc[(csd_mean_grouped['TimePoint'] == 'T1')].add_prefix('T1_mean_').rename(columns={'T1_mean_subject':'subject','T1_mean_TimePoint':'TimePoint'})


# delete remaining columns that are not needed anymore
del act_mean_T0['TimePoint'], act_mean_T1['TimePoint'], act_sd_T0['TimePoint'], act_sd_T1['TimePoint'], csd_mean_T0['TimePoint'], csd_mean_T1['TimePoint'], csd_sd_T0['TimePoint'], csd_sd_T1['TimePoint']
del sleep['Subj_Time']
del screener['Drop.out']

In [7]:
# Init
columns = ['subject','Male', 'Age']
df = pd.DataFrame(columns=columns)
df_agg = pd.DataFrame()

# Build dataframe using subject, male and age. Drop duplicates. 
df['subject'] = ids['subject'].drop_duplicates()
df['Male'] = ids['Male']
df['Age'] = ids['Age']
df['therapy_condition'] = ids['therapy_condition']
df['risk'] = ids['risk']
df['insomnia_subtype'] = isi['insomnia_subtype_true']


# Temporary dataframe to aggregate values from multiple rows in the csv
df_agg['subject'] = ids['subject']

df_agg['T0_IDSSR'] = ids.loc[ids['TimePoint'] == 'T0','IDSSR_score']
df_agg['T1_IDSSR'] = ids.loc[ids['TimePoint'] == 'T1','IDSSR_score']
df_agg['T2_IDSSR'] = ids.loc[ids['TimePoint'] == 'T2','IDSSR_score']
df_agg['T3_IDSSR'] = ids.loc[ids['TimePoint'] == 'T3','IDSSR_score']
df_agg['T4_IDSSR'] = ids.loc[ids['TimePoint'] == 'T4','IDSSR_score']

df_agg['T0_ISI'] = isi.loc[isi['TimePoint'] == 'T0','ISI_score']
df_agg['T1_ISI'] = isi.loc[isi['TimePoint'] == 'T1','ISI_score']
df_agg['T2_ISI'] = isi.loc[isi['TimePoint'] == 'T2','ISI_score']
df_agg['T3_ISI'] = isi.loc[isi['TimePoint'] == 'T3','ISI_score']
df_agg['T4_ISI'] = isi.loc[isi['TimePoint'] == 'T4','ISI_score']

df_agg = df_agg.groupby(df_agg['subject']).aggregate({'T0_IDSSR': 'sum','T1_IDSSR': 'sum','T2_IDSSR': 'sum','T3_IDSSR':'sum','T4_IDSSR': 'sum','T0_ISI': 'sum','T1_ISI': 'sum','T2_ISI': 'sum','T3_ISI': 'sum','T4_ISI': 'sum',})



# Add them to the main dataframe
df = df.merge(df_agg,on='subject')
df = df.merge(ecg_grouped,on='subject')
df = df.merge(screener, on='subject')
df = df.merge(act_mean_T0, on='subject')
df = df.merge(act_sd_T0, on='subject')
df = df.merge(act_mean_T1, on='subject')
df = df.merge(act_sd_T1, on='subject')
df = df.merge(csd_mean_T0, on='subject')
df = df.merge(csd_sd_T0, on='subject')
df = df.merge(csd_mean_T1, on='subject')
df = df.merge(csd_sd_T1, on='subject')
df = df.merge(sleep, on='subject')
df.reset_index(drop=True, inplace=True)
# UNCOMMENT TO USE DIFFERENCE METRICS
# df['T0-T1_IDSSR'] = df['T0_IDSSR'] - df['T1_IDSSR']
# df['T0-T4_IDSSR'] = df['T0_IDSSR'] - df['T4_IDSSR']
# df['T0-T1_ISI'] = df['T0_ISI'] - df['T1_ISI']
# df['T0-T4_ISI'] = df['T0_ISI'] - df['T4_ISI']
# df['T0-T1_Awakenings_TIB'] = df['T0_Awakenings_TIB']-df['T1_Awakenings_TIB']

# UNCOMMENT TO NOT USE DIFFERENT TIME POINTS
df = df.drop(['T2_IDSSR','T3_IDSSR','T4_IDSSR','T2_ISI','T3_ISI','T4_ISI'], axis=1)

# Process column names
df['therapy_condition'].loc[(df['therapy_condition'] == 0)] = 'control'
df['therapy_condition'].loc[(df['therapy_condition'] == 1)] = 'cbti'
df['therapy_condition'].loc[(df['therapy_condition'] == 2)] = 'ct'
df['therapy_condition'].loc[(df['therapy_condition'] == 3)] = 'cbti_ct'
df['therapy_condition'].loc[(df['therapy_condition'] == 4)] = 'tau'
df.rename(columns={'therapy_condition':'therapy'}, inplace=True)
df['risk'].loc[(df['risk'] == 'high_risk')] = 'high'
df['risk'].loc[(df['risk'] == 'low_risk')] = 'low'

# Encode categorical variables
df = pd.get_dummies(df, prefix=None, prefix_sep='_', dummy_na=False, columns=['risk', 'therapy','insomnia_subtype'])

# Exclude controls, comment next line to include
# df = df[df.therapy_control != 1]

df

,subject,Male,Age,T0_IDSSR,T1_IDSSR,T0_ISI,T1_ISI,HR_sleep,SCRN_ISI_score,SCRN_ITQ_ACS_score,SCRN_ITQ_BAS_score,SCRN_ITQ_SHS_score,SCRN_ITQ_RPA_positive_rumination,SCRN_ITQ_RPA_dampening,SCRN_ITQ_RRS_score,SCRN_ITQ_MIPIP_extraversion,SCRN_ITQ_MIPIP_neuroticism,SCRN_ITQ_MIPIP_agreeableness,SCRN_ITQ_PANAS_negative_affect,SCRN_ITQ_PANAS_positive_affect,SCRN_ITQ_TEPS_score,SCRN_ITQ_PI_organization,SCRN_ITQ_PI_perceived_parental_pressure,SCRN_ITQ_PI_rumination,SCRN_ITQ_PI_score_R,SCRN_ITQ_FIRST_score,SCRN_ITQ_PSAS_score,SCRN_ITQ_FSS_score,SCRN_ITQ_CTQ_score,T0_mean_TST_TIB,T0_mean_TIB,T0_mean_SE_TIB,T0_mean_SOW,T0_mean_TST_SOW,T0_mean_Awakenings_TIB,T0_mean_Mean_Temp,T0_mean_Bedtime_Dec_Hr,T0_mean_EyesClosed_Dec_Hr,T0_mean_SleepOpEnd_Dec_Hr,T0_mean_Getuptime_Dec_Hr,...,T1_sd_CSD_WASO_num,T1_sd_CSD_logWASO_num+1,T1_sd_CSD_WASO_min,T1_sd_CSD_logWASO_min+1,T1_sd_CSD_EMA_min,T1_sd_logCSD_EMA_min+1,T1_sd_CSD_Getuptime_Dec_Hr,T1_sd_CSD_Mid-Sleep_Time_Dec_Hr,T1_sd_CSD_Qualit,T1_sd_CSD_WellRested,T1_sd_CSD_BedtimeSleepiness,T1_sd_CSD_OvernightSleepinessChange,T1_sd_CSD_Bedtime_Worry,T1_sd_CSD_Thoughts,T1_sd_CSD_Dreams,T1_sd_CSD_Thoughts-Dreams,T1_sd_CSD_Nap,total_sleep_period_duration_min,N1_perc_of_sleep_period,N2_perc_of_sleep_period,N3_perc_of_sleep_period,R_perc_of_sleep_period,Wake_after_sleep_onset_perc_of_sleep_period,N2_delay_min,SWS_delay_min,R_delay_min,risk_control,risk_high,risk_low,therapy_cbti,therapy_cbti_ct,therapy_control,therapy_ct,therapy_tau,insomnia_subtype_0,insomnia_subtype_1,insomnia_subtype_2,insomnia_subtype_3,insomnia_subtype_4,insomnia_subtype_5
0,1001,0,65,2.0,2.0,0.0,0.0,66.267863,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395.944444,494.666667,0.800565,465.777778,392.944444,16.666667,32.919381,22.380000,22.787778,6.548889,6.623333,...,0.755929,0.171351,2.886751,0.325721,NaN,NaN,0.572418,0.877192,0.487950,0.487950,0.377964,0.690066,0.000000,0.000000,0.755929,0.755929,0.000000,486.5,7.810894,35.868448,26.104830,16.649538,13.566290,4.0,7.5,181.5,1,0,0,0,0,1,0,0,1,0,0,0,0,0
1,1002,0,66,13.0,21.0,11.0,10.0,62.957853,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,461.277778,574.777778,0.802915,560.444444,457.888889,22.888889,32.839046,22.883333,23.153333,8.494444,8.464444,...,2.228602,0.214667,45.249309,0.489125,NaN,NaN,0.661261,0.674675,0.547723,1.095445,0.516398,1.366260,0.408248,0.632456,0.752773,0.752773,0.487950,533.0,7.692308,26.266417,38.649156,17.260788,10.131332,1.5,35.5,96.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
2,1004,0,23,8.0,3.0,1.0,1.0,68.805062,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,449.428571,587.142857,0.764777,570.000000,446.142857,35.857143,34.799896,22.512857,22.784286,8.285714,8.297143,...,0.377964,0.113767,1.889822,0.294056,NaN,NaN,0.998613,0.425469,0.534522,0.577350,0.000000,0.577350,0.755929,0.951190,0.951190,0.000000,0.000000,448.0,3.683036,53.236607,16.629464,19.084821,7.366071,1.5,11.5,126.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
3,1006,0,45,2.0,2.0,0.0,0.0,59.684175,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414.100000,451.600000,0.917681,442.600000,407.600000,11.600000,33.477747,23.374000,23.424000,6.800000,6.900000,...,2.070197,0.179093,2.672612,0.415858,NaN,NaN,1.233591,0.866929,0.786796,0.534522,0.377964,0.755929,1.573592,1.511858,1.380131,1.463850,0.000000,464.5,2.583423,33.692142,22.604952,34.230355,6.889128,1.5,8.5,99.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
4,1007,0,40,23.0,19.0,7.0,6.0,60.126891,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,458.714286,563.571429,0.811520,530.142857,440.142857,20.428571,32.378152,23.570000,24.121429,8.958571,8.962857,...,0.577350,0.087778,66.583281,0.639815,3.535534,0.185969,1.777224,0.681441,0.816497,0.755929,0.755929,0.786796,0.487950,0.377964,1.112697,1.272418,0.377964,468.0,2.884615,56.410256,24.465812,11.645299,4.594017,0.5,12.0,123.0,1,0,0,0,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,..

In [8]:
df.to_csv('data.csv')

### Feature Selection: GridSearch & SelectKBest

**Libraries** 

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb
from xgboost import XGBRegressor
import seaborn as sn

[Heroverweeg hieronder](https://https://medium.com/swlh/feature-importance-hows-and-why-s-3678ede1e58f). XGBoost kan dit zelf, is op zichzelf beter dan lineare regressie, sneller dan random forests, en XGBoost zoekt naar relaties tussen meerdere variabelen. (niet alleen met de target variable). Ook zoekt f_regression naar P_values, wat per variabel individueel kijkt naar de relatie met de target variabel. Dit is niet altijd het geval; soms zijn p-waarden niet significant maar wel in combinaties of interacties. 

Andere opties zijn:

* Random Forest feature importance method
* RFE-Recusrive Feature Elimination (zoekt iteratief naar goede features, en dropped ze ook tussendoor om te zoeken.) Waarschijnlijk niet goed aangezien deze aanneemt dat features niet correleren, maar, zo erg correleren mijn features nou ook weer niet.
* Boruta Feature Selection Algorithms: assumes all features carry *some* useful information rather than a compact subset of features giving minimal errors. (data driven)

Looking for interactions: **[Friedman's H](https://https://blog.macuyiko.com/post/2019/discovering-interaction-effects-in-ensemble-models.html)** or **[EIX](https://https://github.com/ModelOriented/EIX)**


In [ ]:
# Copy the dataframe 
df_a = df.copy()
df_a.head()

# Impute, make sure no features interfere with the selectKbest.  
KNNImputer().fit_transform(df_a)
df_a.replace([np.inf, -np.inf], np.nan, inplace=True)
df_a = df_a.fillna(df_a.mean()) # somehow, some na's survive, just impute these one or two with mean

# Get X and y
X = df_a.loc[:, df_a.columns != 'subject']
y = X.pop('T1_IDSSR')

# print(X.columns)

pipeline = Pipeline(
    [
     ('selector',SelectKBest(f_regression)),
     (('model'),XGBRegressor(objective="reg:squarederror"))
    ]
  )

# TODO: parameters {'':'',}
search = GridSearchCV(
    estimator = pipeline,
    param_grid = {'selector__k':[7,8,9,10,11,12,13,14,15,16,17,18]},
    n_jobs = -1,
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=3

)

pipeline.fit(X,y)
features = pipeline.named_steps['selector']
X.columns[features.get_support()]


search.fit(X,y)
print(search.best_params_)
print(search.best_estimator_)
# print(search.best_estimator_.named_steps['selector'].get_support())

resulting_df = df_a.from_dict(search.cv_results_, orient="columns")
print(resulting_df.columns)

# sn.relplot(data=resulting_df,
# 	kind='line',
# 	x='param_selector__k',
# 	y='mean_test_score',)
# plt.show()

print(X.shape)
select = SelectKBest(score_func=f_regression, k=5)
z = select.fit_transform(X,y)
print("After selecting best 13 features:", z.shape) 
filter = select.get_support()
features = X.columns

print("All features:")
# print(features[filter].values)
features = features[filter].values
print(features)
# Select features using SelectKBest
# features = SelectKBest(score_func=f_regression, k='all')
# features.fit(X_train, y_train)
# X_train_fs = features.transform(X_train)
# X_test_fs = features.transform(X_valid)

# features = X_train.columns[features.get_support()]
# print(X_train_fs)

# **Analysis: XGBoost**



**Libraries** 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb
from xgboost import XGBRegressor
import seaborn as sn

In [ ]:

X = df_a[features]
# X = df_nn[['T0_IDSSR','T0_ISI', 'risk_high','risk_low','therapy_cbti', 'therapy_cbti_ct','therapy_ct', 'therapy_tau', 'SCRN_ISI_score','SCRN_ITQ_FSS_score','SCRN_ITQ_PANAS_negative_affect','SCRN_ITQ_RRS_score','SWS_delay_min','R_perc_of_sleep_period']]
y = df_a.T1_IDSSR
# DMatrix: An optimized data structure for XGBoost. 
data_dmatrix = xgb.DMatrix(data=X, label=y)

# Train, Test, Split
X_train, X_valid, y_train, y_valid =  train_test_split(X, y)

# SimpleImputer(missing_values=np.nan, strategy='mean').fit(X_train)

corrMatrix = X.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()



In [ ]:


# Testing XGBoost
print('%%%%%%% XGBoost Regression:')


# for i in range(len(features.scores)):
#   print('Feature %d: %f' % (i, features.scores_[i]))


xg_reg = XGBRegressor(objective="reg:squarederror")
xg_reg.fit(X_train, y_train)
predictions = xg_reg.predict(X_valid)
print("R2: " + str(r2_score(y_valid, predictions)))
print("RMSE: " + str(np.sqrt(mean_squared_error(y_valid, predictions))))
xgb.plot_importance(xg_reg)
plt.show()
print(xg_reg.get_xgb_params())
print('%%%%%%% With Cross-validation:')
# With Cross Validation
params = {"objective":"reg:squarederror"}
xg_reg_cross = xgb.train(dtrain=data_dmatrix, params=params)
xgb_cv = xgb.cv(dtrain=data_dmatrix, params=params, metrics="mae", as_pandas=True, nfold=6) 
xgb.plot_importance(xg_reg_cross)
plt.show()

print(xgb_cv)

